In [1]:
import torch
print(torch.cuda.is_available())

from stable_baselines3 import PPO
import gymnasium as gym
from continuous_env.robot_obstacles import RobotObstacles

gym.register(
    id="RobotObstacleEnv-v1",
    entry_point=RobotObstacles,
    max_episode_steps=10000,
)

True


In [ ]:
from stable_baselines3.common.vec_env import VecTransposeImage, SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback
import multiprocessing as mp
mp.set_start_method('spawn', force=True)

TIMESTEPS = 20_000_000

def make_env():
    return RobotObstacles()

log_path = "./logs"
num_envs = 4
envs = SubprocVecEnv([make_env for _ in range(num_envs)])
env = VecTransposeImage(envs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PPO("CnnPolicy", env, device=device, tensorboard_log=log_path)

checkpoint_callback = CheckpointCallback(save_freq=1_000_000, save_path="./checkpoints/", name_prefix="ppo_model")
print("Learning")
model.learn(total_timesteps=TIMESTEPS, callback=checkpoint_callback)

Learning


In [ ]:
obs, _info = env.reset()
avg_reward = 0
num_envs = 1
for _ in range(10000):
    action, _states = model.predict(obs)
    obs, reward, term, trunc, info = env.step(action)
    done = term or trunc
    if done:
        num_envs += 1
        avg_reward += reward
        obs, _info = env.reset()

avg_reward /= num_envs
avg_reward

In [3]:
# model.save("weights/ppo_model")